In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.distributions import Categorical
import numpy as np

# Importações de módulos locais (presume-se que existam no diretório do projeto)
# rl_env: Contém a definição do ambiente de simulação (PortfolioEnv) e as redes neurais (PolicyMLP, ValueMLP)
# app: Contém dados financeiros pré-processados e funções auxiliares
from rl_env import PortfolioEnv, PolicyMLP, ValueMLP
from app import (ret, r_media, v_media, dd, regime_ids, cluster_ids, NUM_ATIVOS)
from app import aplicar_acao_portfolio

# ==============================================================================
# CONFIGURAÇÃO DE HIPERPARÂMETROS E DIMENSÕES
# ==============================================================================

# Define quantas ações são possíveis por ativo (ex: 2 poderia ser 'comprar' ou 'vender',
# ou aumentar/diminuir exposição).
AÇÕES_POR_ATIVOS = 2

# O espaço de ação total é o número de ativos multiplicado pelas ações possíveis por ativo.
# Isso sugere um output 'flat' onde o modelo escolhe um índice que representa (Ativo X, Ação Y).
num_actions = NUM_ATIVOS * AÇÕES_POR_ATIVOS

# Definição da dimensão do estado (input da rede neural).
# A soma abaixo (NUM_ATIVOS + 3 + NUM_ATIVOS) indica que o estado é composto por:
# 1. Alocação atual (pesos) dos ativos (NUM_ATIVOS)
# 2. Informações de mercado/contexto (3 variáveis, ex: regime, volatilidade, etc)
# 3. Alguma outra métrica por ativo ou retornos passados (NUM_ATIVOS)
state_dim = NUM_ATIVOS + 3 + NUM_ATIVOS

# ==============================================================================
# INICIALIZAÇÃO DOS MODELOS E OTIMIZADORES
# ==============================================================================


# Inicializa a rede da Política (Actor): Recebe o estado e retorna probabilidades de ação.
policy = PolicyMLP(state_dim, num_actions)

# Inicializa a rede de Valor (Critic): Recebe o estado e estima o valor (V) daquele estado.
value = ValueMLP(state_dim)

def init_weights(m):
# Checa se o módulo é uma camada totalmente conectada
    if isinstance(m, nn.Linear):
# Define os pesos dessa camada com a inicialização de Xavier (Glorot
        nn.init.xavier_uniform_(m.weight)
# Atribui zero aos vieses
        nn.init.zeros_(m.bias)

policy.apply(init_weights)
value.apply(init_weights)


# Configura os otimizadores (Adam) para ajustar os pesos das redes.
# lr=1e-4 é a taxa de aprendizado (learning rate).
optimizerP = optim.Adam(policy.parameters(), lr=1e-4)
optimizerV = optim.Adam(value.parameters(), lr=1e-4)

# ==============================================================================
# INICIALIZAÇÃO DO AMBIENTE
# ==============================================================================

# Cria a instância do ambiente de portfólio passando os dados financeiros necessários.
env = PortfolioEnv(ret, r_media, v_media, dd, regime_ids, cluster_ids)

# ==============================================================================
# LOOP DE TREINAMENTO (EPISÓDIOS)
# ==============================================================================

episodios = 5   # Número de episódios para rodar (valor baixo apenas para debug/teste)

for ep in range(episodios):
    # Reseta o ambiente para o estado inicial no começo de cada episódio
    state = env.reset()
    
    # Converte o estado (numpy array) para Tensor do PyTorch e adiciona dimensão de batch (unsqueeze)
    state = torch.FloatTensor(state).unsqueeze(0)

    done = False        # Flag para controlar o fim do episódio
    total_reward = 0    # Acumulador de recompensa do episódio

    # Loop principal de interação passo-a-passo (timesteps) dentro do episódio
    while not done:

        # --- 1. Escolha da Ação (Actor) ---
        
        
        logits = policy(state)

        
        if torch.isnan(logits).any() or not torch.isfinite(logits).all():
            print("DEU RUIM: logits inválidos (NaN/Inf)")
            print("state:", state)
            print("logits:", logits)
            break
        
        
        m = Categorical(logits=logits)
        
        # Amostra uma ação baseada na distribuição (exploração estocástica)
        action = m.sample()
        
        # Calcula o log da probabilidade da ação escolhida (necessário para calcular a loss depois)
        log_prob = m.log_prob(action)

        # --- 2. Execução no Ambiente ---

        # Função auxiliar que traduz o índice da ação (int) para o novo vetor de pesos do portfólio
        novos_pesos = aplicar_acao_portfolio(env.pesos, action.item())

        # O ambiente dá um passo com os novos pesos e retorna:
        # next_state: O novo estado do mercado/portfólio
        # reward: A recompensa imediata (ex: retorno financeiro, Sharpe ratio, etc)
        # done: Se o episódio acabou (ex: fim dos dados históricos ou falência)
        next_state, reward, done = env.step(novos_pesos)

        # Prepara o próximo estado para processamento pelo PyTorch
        next_state_tensor = torch.FloatTensor(next_state).unsqueeze(0)

        # --- 3. Cálculo do Valor e Advantage (Critic) ---

        # Estima o valor do estado ATUAL V(s)
        V = value(state)
        
        # Estima o valor do PRÓXIMO estado V(s'). .detach() é usado porque não queremos
        # propagar gradientes através desta estimativa alvo (target).
        V_next = value(next_state_tensor).detach()

        # Calcula o Advantage (Vantagem) usando TD Error (Temporal Difference de 1 passo):
        # A = Reward + (gamma * V_next) - V_current
        # gamma aqui é 0.99 (fator de desconto). 
        # (1 - int(done)) garante que se o episódio acabou, o valor futuro é 0.
        advantage = reward + (0.99 * V_next * (1 - int(done))) - V

        # --- 4. Cálculo das Perdas (Losses) e Backpropagation ---

        # Policy Loss (Actor Loss): Maximizar log_prob * advantage.
        # Como o otimizador minimiza, usamos o sinal negativo (-).
        # .detach() no advantage é crucial para não atualizar o Critic via loss do Actor.
        policy_loss = -(log_prob * advantage.detach())

        # Value Loss (Critic Loss): Erro quadrático médio entre a estimativa de valor e o alvo (TD Target).
        # Basicamente: queremos que V(s) preveja corretamente Reward + V(s').
        value_loss = advantage.pow(2)

        # Atualização da rede da Política (Actor)
        optimizerP.zero_grad()      # Zera gradientes anteriores
        policy_loss.backward()      # Calcula gradientes
        optimizerP.step()           # Atualiza pesos

        # Atualização da rede de Valor (Critic)
        optimizerV.zero_grad()
        value_loss.backward()
        optimizerV.step()

        # --- 5. Atualização de Estado ---
        
        # O próximo estado vira o estado atual para a próxima iteração
        state = next_state_tensor
        total_reward += reward

    # Fim do episódio
    print(f"Episódio {ep} -> reward total = {total_reward:.5f}")

Arquivo encontrado! Carregando dados locais...
            BOVA11.SA  SMAL11.SA  IVVB11.SA  PETR4.SA   VALE3.SA  ITUB4.SA  \
Date                                                                         
2015-01-01        NaN        NaN        NaN       NaN        NaN       NaN   
2015-01-02  47.259998  52.020000  55.799999  2.572509  10.505502  9.372999   
2015-01-03        NaN        NaN        NaN       NaN        NaN       NaN   
2015-01-04        NaN        NaN        NaN       NaN        NaN       NaN   
2015-01-05  46.320000  50.549999  55.750000  2.352637  10.347521  9.420101   

                   SPY        QQQ         IWM        EEM         GLD  \
Date                                                                   
2015-01-01         NaN        NaN         NaN        NaN         NaN   
2015-01-02  171.093704  94.906532  103.315140  30.789457  114.080002   
2015-01-03         NaN        NaN         NaN        NaN         NaN   
2015-01-04         NaN        NaN         NaN 

c:\Users\olazu\OneDrive\Documentos\Python\portfolioDeAtivos\portfolioDeAtivos\teste.py:154: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  retornos = precos.pct_change().fillna(0)


state shape: torch.Size([1, 37])
action_probs shape: torch.Size([1, 34])
sum probs: 1.0000001192092896
Ep 0 | reward -0.00004 | regime neutro
Ep 200 | reward -0.00732 | regime neutro
Ep 400 | reward -0.02160 | regime alta_vol
Ep 600 | reward 0.00132 | regime bull
Ep 800 | reward 0.00834 | regime bull
Ep 1000 | reward 0.00192 | regime bull
Ep 1200 | reward 0.00355 | regime alta_vol
Ep 1400 | reward 0.00704 | regime alta_vol
Ep 1600 | reward -0.00582 | regime alta_vol
Ep 1800 | reward 0.00522 | regime neutro
Ep 2000 | reward 0.00164 | regime alta_vol
Ep 2200 | reward 0.01999 | regime bull
Ep 2400 | reward 0.00675 | regime alta_vol
Ep 2600 | reward -0.01032 | regime alta_vol
Ep 2800 | reward -0.01639 | regime bear
Ep 3000 | reward -0.02224 | regime bear
Ep 3200 | reward 0.00079 | regime bear
Ep 3400 | reward 0.00215 | regime alta_vol
Ep 3600 | reward 0.01224 | regime bull
Ep 3800 | reward -0.00771 | regime alta_vol
Episódio 0 -> reward total = -3.72957
Episódio 1 -> reward total = -3.0385